In [1]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
import librosa.display
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 15

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 21
channel = 1
epochs = 100
batch_size = 100
verbose = 1
num_classes = 15

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Using TensorFlow backend.
Saving vectors of label - '99WNoise': 100%|████████████████████████████████████████| 2759/2759 [00:28<00:00, 97.67it/s]


In [5]:
def get_model():
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(16, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(44, (2, 2), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(20, activation='relu'))
    #model.add(Dense(3, activation='softmax'))
    
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath, max_len=feature_dim_2)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [6]:
epochs = 50
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3585 samples, validate on 2390 samples
Epoch 1/50
3585/3585 [==============================] - ETA: 4s - loss: 7.9030 - accuracy: 0.04 - ETA: 1s - loss: 4.6227 - accuracy: 0.14 - ETA: 0s - loss: 3.8140 - accuracy: 0.21 - ETA: 0s - loss: 3.4157 - accuracy: 0.25 - ETA: 0s - loss: 3.1737 - accuracy: 0.28 - ETA: 0s - loss: 3.0091 - accuracy: 0.31 - ETA: 0s - loss: 2.8480 - accuracy: 0.33 - ETA: 0s - loss: 2.7377 - accuracy: 0.34 - ETA: 0s - loss: 2.6446 - accuracy: 0.36 - 1s 222us/step - loss: 2.5878 - accuracy: 0.3662 - val_loss: 1.8610 - val_accuracy: 0.4410
Epoch 2/50
3585/3585 [==============================] - ETA: 0s - loss: 1.8725 - accuracy: 0.45 - ETA: 0s - loss: 1.8721 - accuracy: 0.46 - ETA: 0s - loss: 1.7850 - accuracy: 0.48 - ETA: 0s - loss: 1.7705 - accuracy: 0.47 - ETA: 0s - loss: 1.7433 - accuracy: 0.48 - ETA: 0s - loss: 1.7345 - accuracy: 0.47 - ETA: 0s - loss: 1.7035 - accuracy: 0.48 - ETA: 0s - loss: 1.6970 - accuracy: 0.48 - ETA: 0s - loss: 1.6853 - accuracy: 0

3585/3585 [==============================] - ETA: 0s - loss: 0.0279 - accuracy: 1.00 - ETA: 0s - loss: 0.0315 - accuracy: 0.99 - ETA: 0s - loss: 0.0401 - accuracy: 0.99 - ETA: 0s - loss: 0.0371 - accuracy: 0.99 - ETA: 0s - loss: 0.0349 - accuracy: 0.99 - ETA: 0s - loss: 0.0438 - accuracy: 0.98 - ETA: 0s - loss: 0.0474 - accuracy: 0.98 - ETA: 0s - loss: 0.0510 - accuracy: 0.98 - ETA: 0s - loss: 0.0476 - accuracy: 0.98 - 1s 181us/step - loss: 0.0492 - accuracy: 0.9852 - val_loss: 0.0395 - val_accuracy: 0.9904
Epoch 32/50
3585/3585 [==============================] - ETA: 0s - loss: 0.1225 - accuracy: 0.97 - ETA: 0s - loss: 0.0461 - accuracy: 0.98 - ETA: 0s - loss: 0.0348 - accuracy: 0.98 - ETA: 0s - loss: 0.0388 - accuracy: 0.98 - ETA: 0s - loss: 0.0466 - accuracy: 0.98 - ETA: 0s - loss: 0.0441 - accuracy: 0.98 - ETA: 0s - loss: 0.0510 - accuracy: 0.98 - ETA: 0s - loss: 0.0478 - accuracy: 0.98 - ETA: 0s - loss: 0.0512 - accuracy: 0.98 - 1s 173us/step - loss: 0.0502 - accuracy: 0.9833 - va

In [7]:
model.save('model.h5')

## Prediction

In [6]:
print(predict('./data/06Vremya/a00010.wav', model=model))

06Vremya


In [7]:
print(predict('./data/04Temperatyry/a00025.wav', model=model))

04Temperatyry
